In [ ]:
!nvidia-smi
git_proxy="https://github.moeyy.xyz/"

%cd /root/
!git clone {git_proxy}https://github.com/innnky/so-vits-svc -b 32k

In [ ]:
!apt update
!apt upgrade
!apt install libsndfile-dev

In [ ]:
#@title 安装依赖
%cd /root/so-vits-svc
!pip install pyworld praat-parselmouth

In [ ]:
#@title 下载必要模型文件
!wget -P hubert/ {git_proxy}https://github.com/bshall/hubert/releases/download/v0.1/hubert-soft-0d54a1f4.pt


In [ ]:
ZIP_NAME="mikisayaka"
!mkdir /root/so-vits-svc/dataset_raw/mikisayaka
!unzip -d /root/so-vits-svc/dataset_raw {ZIP_NAME}.zip

In [ ]:
#@title 重采样到32k/48k，根据你选择克隆的github仓库分支自动确定
!python resample.py

In [ ]:
#@title 划分训练集 生成配置文件
!python preprocess_flist_config.py

In [ ]:
#@title 生成hubert和f0
!python preprocess_hubert_f0.py


# 训练

In [ ]:
#@title  选择是否将训练后的模型保存到谷歌云盘 和 是否使用预模型
#@markdown ##选择你克隆的github仓库的分支和你最开始选的保持一致
Clone = "32k" #@param ["32k","48k"]

#@markdown **首次训练下载预模型 之后继续训练则使用自己保存的记录点，无需再下载**

#@markdown **使用预模型，下面打钩自动下载并启用**
pre_pth = True #@param {type:"boolean"}
if pre_pth:
    !wget -P logs/"{Clone}"/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/G_0.pth
    !wget -P logs/"{Clone}"/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/D_0.pth


In [ ]:
#@title  开始训练
#@markdown ##选择你克隆的github仓库的分支和你最开始选的保持一致
Clone = "32k" #@param ["32k","48k"]

#@markdown **开始训练**

#@markdown **启用tensorboard可视化数据**
tensorboard_on = True #@param {type:"boolean"}
if tensorboard_on:
    %load_ext tensorboard
    %tensorboard --logdir logs/"{Clone}"

!python train.py -c configs/config.json -m "{Clone}"
# python train.py -c configs/config.json -m 32k
